## Existing Agent Executor

In [1]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import RunnablePassthrough
from langgraph.prebuilt.tool_executor import ToolExecutor
from langgraph.prebuilt.agent_executor import create_agent_executor

from langgraph.graph import END, Graph

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)
tool_executor = ToolExecutor(tools)
chain = create_agent_executor(agent_runnable, tool_executor)

/Users/harrisonchase/workplace/langchain/libs/core/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
chain.invoke({"input": "what is the weather in sf"})

{'input': 'what is the weather in sf',
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query":"weather in San Francisco"}'}})]),
   [{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/january/',
     'content': 'Best time to go to San Francisco? Weather in San Francisco in january 2024  How was the weather last january? Here is the day by day recorded weather in San Francisco in january 2023:  Seasonal average climate and temperature of San Francisco in january  8% 46% 29% 12% 8% Evolution of daily average temperature and precipitation in San Francisco in januaryWeather in San Francisco in january 2024. The weather in San Francisco in january comes 

## Agent Messages Executor

In [3]:
from langchain_core.agents import AgentAction
from langchain_core.messages import FunctionMessage, HumanMessage, SystemMessage
from langchain.tools.render import format_tool_to_openai_function
import json
from langchain.chat_models import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.prebuilt.tool_executor import ToolExecutor
from langgraph.prebuilt.agent_messages_executor import create_agent_messages_executor
tools = [TavilySearchResults(max_results=1)]
model = ChatOpenAI().bind_functions([format_tool_to_openai_function(t) for t in tools])

In [4]:
def call_model(messages):
    response = model.invoke(messages)
    return messages + [response]


def exit(messages):
    last_message = messages[-1]
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    else:
        return "function"

tool_executor = ToolExecutor(tools)
def call_tool(messages):
    last_message = messages[-1]
    action = AgentAction(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
        log="",
    )
    response = tool_executor.execute(action)
    function_message = FunctionMessage(content=str(response[0][1]), name=action.tool)
    return messages + [function_message]

In [5]:
chain = create_agent_messages_executor(call_model, call_tool, exit)

In [6]:
messages = [HumanMessage(content="what is the weather in sf")]
for s in chain.stream(messages):
    print(list(s.values())[0])
    print("----")

[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}})]
----
[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}}), FunctionMessage(content="[{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/january/', 'content': 'Best time to go to San Francisco? Weather in San Francisco in january 2024  How was the weather last january? Here is the day by day recorded weather in San Francisco in january 2023:  Seasonal average climate and temperature of San Francisco in january  8% 46% 29% 12% 8% Evolution of daily average temperature and precipitation in San Francisco in januaryWeather in San Francisco in january 2024. The weather in San Francisco in januar

### Human in the Loop

#### Require confirmation

In [7]:
def call_tool(messages):
    last_message = messages[-1]
    action = AgentAction(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
        log="",
    )
    response = input(prompt=f"[y/n] Okay to call this tool? {action}")
    if response == "n":
        raise ValueError
    response = tool_executor.execute(action)
    function_message = FunctionMessage(content=str(response[0][1]), name=action.tool)
    return messages + [function_message]

In [8]:
chain = create_agent_messages_executor(call_model, call_tool, exit)

In [9]:
messages = [HumanMessage(content="what is the weather in sf")]
for s in chain.stream(messages):
    print(list(s.values())[0])
    print("----")

[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}})]
----


[y/n] Okay to call this tool? tool='tavily_search_results_json' tool_input={'query': 'weather in San Francisco'} log='' 


[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}}), FunctionMessage(content="[{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/january/', 'content': 'Best time to go to San Francisco? Weather in San Francisco in january 2024  How was the weather last january? Here is the day by day recorded weather in San Francisco in january 2023:  Seasonal average climate and temperature of San Francisco in january  8% 46% 29% 12% 8% Evolution of daily average temperature and precipitation in San Francisco in januaryWeather in San Francisco in january 2024. The weather in San Francisco in january comes from statistical datas on the past years. You can view the weather statistics the entire month, but also by using the tabs for the beginning, the middle and the end of the month. ... 12-01-2023 50°F to 59°F. 1

In [13]:
def call_tool(messages):
    last_message = messages[-1]
    action = AgentAction(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
        log="",
    )
    response = input(prompt=f"[y/n] Okay to call this tool? {action}")
    if response == "**EXIT**":
        raise ValueError
    elif response:
        print("foo")
        action.tool_input = response
    response = tool_executor.execute(action)
    function_message = FunctionMessage(content=str(response[0][1]), name=action.tool)
    return messages + [function_message]

In [14]:
chain = create_agent_messages_executor(call_model, call_tool, exit)

In [16]:
messages = [HumanMessage(content="what is the weather in sf")]
for s in chain.stream(messages):
    print(list(s.values())[0])
    print("----")

[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}})]
----


[y/n] Okay to call this tool? tool='tavily_search_results_json' tool_input={'query': 'weather in San Francisco'} log='' {'query': 'current weather in SF'}


foo
[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}}), FunctionMessage(content="[{'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/', 'content': 'San Francisco Weather in January  you can find all information about the weather in San Francisco in January:  San Francisco weather in January San Francisco weather by month // weather averages 9.6 (49.2) 6.2 (43.2) 14 (57.3) 113  San Francisco weather in January // weather averages Airport close to San FranciscoData: 1991 - 2021 Min. Temperature °C (°F), Max. Temperature °C (°F), Precipitation / Rainfall mm (in), Humidity, Rainy days. Data: 1999 - 2019: avg. Sun hours San Francisco weather and climate for further months San Francisco in February San Francisco in March San Francisco in April San Francisco in May San F

## Respond in a specific format

In [20]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

In [21]:
class Answer(BaseModel):
    """Final Response"""
    temp: int = Field(description="current temperature, in Farenheit")
    source: List[str] = Field(description="URLs to go to to learn more info")

In [22]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
model = ChatOpenAI().bind_functions([format_tool_to_openai_function(t) for t in tools] + [convert_pydantic_to_openai_function(Answer)])

In [23]:
def call_model(messages):
    response = model.invoke(messages)
    return messages + [response]


def exit(messages):
    last_message = messages[-1]
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    elif "function_call" in last_message.additional_kwargs and last_message.additional_kwargs["function_call"]["name"] == "Answer":
        return "end"
    else:
        return "function"

In [24]:
chain = create_agent_messages_executor(call_model, call_tool, exit)

In [25]:
messages = [HumanMessage(content="what is the weather in sf")]
for s in chain.stream(messages):
    print(list(s.values())[0])
    print("----")

[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "current weather in San Francisco"\n}'}})]
----


[y/n] Okay to call this tool? tool='tavily_search_results_json' tool_input={'query': 'current weather in San Francisco'} log='' 


[HumanMessage(content='what is the weather in sf'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "current weather in San Francisco"\n}'}}), FunctionMessage(content="[{'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/', 'content': 'San Francisco Weather in January  you can find all information about the weather in San Francisco in January:  San Francisco weather in January San Francisco weather by month // weather averages 9.6 (49.2) 6.2 (43.2) 14 (57.3) 113  San Francisco weather in January // weather averages Airport close to San FranciscoData: 1991 - 2021 Min. Temperature °C (°F), Max. Temperature °C (°F), Precipitation / Rainfall mm (in), Humidity, Rainy days. Data: 1999 - 2019: avg. Sun hours San Francisco weather and climate for further months San Francisco in February San Francisco in March San Francisco in April San Francisco in May S

## Tool State

In [1]:
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field

In [2]:
class IntSchema(BaseModel):
    num: int
    ls: dict

    @classmethod
    def schema(cls):
        schema = super().schema()
        properties = schema.get('properties', {})
        properties.pop('ls', None)  # Remove the hidden attribute
        return schema

In [3]:
@tool(args_schema=IntSchema)
def add_int(num, ls):
    """Call this to add number to the list."""
    ls["foo"].append(num)
    print(ls)
    return "Done!"

In [4]:
IntSchema.schema()["properties"]

{'num': {'title': 'Num', 'type': 'integer'}}

In [5]:
from langchain_core.agents import AgentAction
from langchain_core.messages import FunctionMessage, HumanMessage, SystemMessage
from langchain.tools.render import format_tool_to_openai_function
import json
from langchain.chat_models import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.prebuilt.tool_executor import ToolExecutor
from langgraph.prebuilt.executor import create_executor
tools = [add_int]
model = ChatOpenAI().bind_functions([format_tool_to_openai_function(t) for t in tools])

/Users/harrisonchase/workplace/langchain/libs/core/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
ls = {"foo": []}
def call_model(messages):
    response = model.invoke(messages)
    return messages + [response]


def exit(messages):
    last_message = messages[-1]
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"

tool_executor = ToolExecutor(tools)
def call_tool(messages):
    last_message = messages[-1]
    agent_action = AgentAction(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
        log="",
    )
    agent_action.tool_input["ls"] = ls
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    # Call that tool on the input
    observation = tool_to_use.invoke(agent_action.tool_input)
    function_message = FunctionMessage(content=str(observation), name=agent_action.tool)
    return messages + [function_message]

In [7]:
chain = create_executor(call_model, call_tool, exit)

In [8]:
ls

{'foo': []}

In [9]:
messages = [HumanMessage(content="add the number one to the list")]
for s in chain.stream(messages):
    print(list(s.values())[0])
    print("----")

[HumanMessage(content='add the number one to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 1\n}'}})]
----
{'foo': [1]}
[HumanMessage(content='add the number one to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 1\n}'}}), FunctionMessage(content='Done!', name='add_int')]
----
[HumanMessage(content='add the number one to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 1\n}'}}), FunctionMessage(content='Done!', name='add_int'), AIMessage(content='The number one has been added to the list.')]
----
[HumanMessage(content='add the number one to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 1\n}'}}), FunctionMessage(content='Done!', name='add_int'), AIMessage(content='The number one has been added to the list.')]
----


In [10]:
ls

{'foo': [1]}

In [11]:
messages = [HumanMessage(content="add the number 3 to the list")]
for s in chain.stream(messages):
    print(list(s.values())[0])
    print("----")

[HumanMessage(content='add the number 3 to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 3\n}'}})]
----
{'foo': [1, 3]}
[HumanMessage(content='add the number 3 to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 3\n}'}}), FunctionMessage(content='Done!', name='add_int')]
----
[HumanMessage(content='add the number 3 to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 3\n}'}}), FunctionMessage(content='Done!', name='add_int'), AIMessage(content='The number 3 has been added to the list.')]
----
[HumanMessage(content='add the number 3 to the list'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_int', 'arguments': '{\n  "num": 3\n}'}}), FunctionMessage(content='Done!', name='add_int'), AIMessage(content='The number 3 has been added to the list.')]
----


In [12]:
ls

{'foo': [1, 3]}

## Reflexion Agent

In [3]:
from langchain.agents import AgentExecutor, BaseMultiActionAgent, Tool
from langchain.schema import AgentAction, AgentFinish
from langchain_core.language_models.chat_models import BaseChatModel
from langchain.chains import LLMChain

from langchain.globals import set_llm_cache

from dotenv import load_dotenv

from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.cache import SQLiteCache

from langchain_core.output_parsers import BaseOutputParser

from langchain.prompts.chat import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.pydantic_v1 import BaseModel
import os

from langchain.agents import AgentType, initialize_agent, load_tools

set_llm_cache(SQLiteCache(database_path=".langchain.db"))


llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=2000,
    max_retries=100,
    model="gpt-4-1106-preview",
)

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

NEXT_STEP_TEMPLATE = """You are expert researcher trying answer a question ~250 words. You are asked to answer the following question: {question}

The way you are going to answer the question is as follows:

1. Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        _ You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
2. Reflect and critique your answer. Specifically, you should:
    - Think about what is missing from your answer.
    - Think about what is superfluous in your answer.
    - Think about what search query you should use next to improve your answer.
  Give your answer in exactly 2 parts. The first should address what is missing from your answer. The second should address what could be removed from your answer. Your should be VERY harsh as we really want to improve the answer.
3. Give the search query you came up with to improve your answer.

Previous steps: 

{previous_steps}

===

Format your answer as follows:

Revised answer: [give your revised answer based on the previous critique and new information from the search engine then the "References" section]
Critique: [give your harsh critique of your revised answer in 2 parts: what is missing and what is superfluous]
Search query: [give the new search query you came up with to enter into the search engine to improve your answer. If you have more than one, make sure they are comma separated and in quotes]

SAY NOTHING else please."""

INITIAL_ANSWER_TEMPLATE = """You are expert researcher trying answer a question ~250 words. You are asked to answer the following question: {question}

The way you are going to answer the question is as follows:

1. Give a detailed in ~250 words.
2. Reflect and critique your answer. Specifically, you should:
    - Think about what is missing from your answer.
    - Think about what is superfluous in your answer.
    - Think about what search query you should use next to improve your answer.
  Give your answer in exactly 2 parts. The first should address what is missing from your answer. The second should address what could be removed from your answer. Your should be VERY harsh as we really want to improve the answer.
3. Give the search query you came up with to improve your answer.

===

Format your answer as follows:

Answer: [give your initial answer]
Critique: [give your harsh critique of your answer in 2 parts: what is missing and what is superfluous]
Search query: [give the search query you came up with to improve your answer. If you have more than one, make sure they are comma separated and in quotes]

SAY NOTHING else please."""


class ReflexionStep(BaseModel):
    """A single step in the reflexion process."""

    answer: str
    critique: str
    search_query: str

    def __str__(self):
        return f"Answer: {self.answer}\nCritique: {self.critique}\nSearch query: {self.search_query}"

def _parse_reflexion_step(output: str) -> tuple[str, str, str]:
    # find answer using .split()
    if ("Answer:" not in output and "Revised answer:" not in output) or not "Critique:" in output or not "Search query:" in output:
        raise ValueError(f"The output is not formatted correctly. Output: {output}")
    if "Answer:" in output:
        answer = output.split("Answer:")[1].split("Critique:")[0].strip()
    else:
        answer = output.split("Revised answer:")[1].split("Critique:")[0].strip()
    critique = output.split("Critique:")[1].split("Search query:")[0].strip()
    search_query = output.split("Search query:")[1].strip()
    return answer, critique, search_query

class ReflexionStepParser(BaseOutputParser[ReflexionStep]):
    """Parser for the reflexion step."""

    def parse(self, output: str) -> ReflexionStep:
        """Parse the output."""
        # try to find answer or initial answer
        answer, critique, search_query = _parse_reflexion_step(output)
        return ReflexionStep(
            answer=answer, critique=critique, search_query=search_query
        )

In [4]:
initial_chain = RunnablePassthrough.assign(
    agent_outcome = ChatPromptTemplate.from_template(INITIAL_ANSWER_TEMPLATE) | llm | ReflexionStepParser() | (lambda x: AgentAction(
                    tool="tavily_search_results_json",
                    tool_input=x.search_query,
                    log=str(x),
                ))
)

def prep_next(inputs):
    intermediate_steps = inputs["intermediate_steps"]
    previous_steps = list[str]()

    for i, (action, observation) in enumerate(intermediate_steps, start=1):
        last_step_str = f"""Step {i}:

{action.log}

Search output for "{action.tool_input}":

{observation}"""
        previous_steps.append(last_step_str)

    previous_steps_str = "\n\n".join(previous_steps)
    inputs["previous_steps"] = previous_steps_str
    return inputs
    
next_chain = RunnablePassthrough.assign(
    agent_outcome = prep_next | ChatPromptTemplate.from_template(NEXT_STEP_TEMPLATE) | llm | ReflexionStepParser() | (lambda x: AgentAction(
                tool="tavily_search_results_json",
                tool_input=x.search_query,
                log=str(x),
            ))
)

def finish(inputs):
    intermediate_steps = inputs["intermediate_steps"]
    last_action, _ = intermediate_steps[-1]
    last_step_str = last_action.log
    # extract answer
    answer, _, _ = _parse_reflexion_step(last_step_str)

    first_action, _ = intermediate_steps[0]
    first_step_str = first_action.log
    # extract answer
    initial_answer, _, _ = _parse_reflexion_step(first_step_str)

    return AgentFinish(
        log="Reached max steps.",
        return_values={"output": answer, "initial_answer": initial_answer},
    )


def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    observation = {t.name: t for t in tools}[agent_action.tool].invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data


In [5]:
workflow = Graph()

# add actors
workflow.add_node("initial", initial_chain)
workflow.add_node("next", next_chain)
workflow.add_node("finish", finish)
workflow.add_node("tools", execute_tools)

# Enter with initial actor, then loop through tools -> next steps until finished
workflow.set_entry_point('initial')

workflow.add_edge('initial', 'tools')
workflow.add_conditional_edges(
    'tools',
    lambda x: "exit" if len(x['intermediate_steps']) >= 2 else "continue",
    {
        "continue": 'next',
        "exit": 'finish'
    }
)
workflow.add_edge('next', 'tools')
workflow.set_finish_point('finish')

chain = workflow.compile()

chain.invoke({"question": "what is the weather in sf", "intermediate_steps": []})

AgentFinish(return_values={'output': "The current weather in San Francisco can be accessed through various weather reporting services, which provide real-time temperature, humidity, wind, and chances of precipitation [1]. Historically, San Francisco experiences a mild, Mediterranean climate with average temperatures ranging from the low 50s to the mid-60s Fahrenheit. The city's unique topography creates microclimates, leading to significant weather variations across different neighborhoods. San Francisco's summers are notably cooler compared to other Californian cities, largely due to the cold California Current and persistent fog, especially in June and July. Winters are mild and the wettest months span from November to March, with an annual rainfall average of approximately 23 inches. Wind is a prominent feature, with spring being particularly windy. For historical weather extremes and average wind speeds, additional specific data can be sought from climatological records.\n\nReferen

In [ ]:
## Plan and Execute

from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple


class PlanExecute(BaseModel):

    plan: List[str] = []
    past_steps: List[Tuple] = []
    response: str = ""

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.tools import tool

class Plan(BaseModel):
    """Plan to follow in future"""
    steps: List[str] = Field(description="different steps to follow, should be in sorted order")

planner_prompt = ChatPromptTemplate.from_template("""For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

{objective}""")
planner = create_structured_output_runnable(Plan, ChatOpenAI(model="gpt-4-1106-preview", temperature=0), planner_prompt)

planner.invoke({'objective': 'what is leo dicaprios gf age raised to .23'})

@tool
def search(query:str):
    """Get a response from google"""
    return 25

@tool
def math(equation: str):
    """Solve a math equation"""
    return .34

tools = [search, math]

from langchain import hub
from langchain.agents import create_openai_functions_agent
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")
# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish


# Define the agent
# Note that here, we are using `.assign` to add the output of the agent to the dictionary
# This dictionary will be returned from the node
# The reason we don't want to return just the result of `agent_runnable` from this node is
# that we want to continue passing around all the other inputs
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def action(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data.pop('agent_outcome')
    # Get the tool to use
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    # Call that tool on the input
    observation = tool_to_use.invoke(agent_action.tool_input)
    # We now add in the action and the observation to the `intermediate_steps` list
    # This is the list of all previous actions taken and their output
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

def plan(inputs):
    inputs['state'] = PlanExecute(plan=planner.invoke(inputs).steps)
    inputs['input'] = inputs['state'].plan[0]
    inputs['intermediate_steps'] = []
    return inputs

from langchain.chains.openai_functions import create_openai_fn_runnable
class Response(BaseModel):
    """Response to user."""
    response: str

replanner_prompt = ChatPromptTemplate.from_template("""For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{objective}

Your original plan was this:
{plan}

You have currently done the follow steps:
{steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan.""")


replanner = create_openai_fn_runnable([Plan, Response], ChatOpenAI(model="gpt-4-1106-preview", temperature=0), replanner_prompt)

replanner.invoke({"objective": "look up the temperature", "plan": "look up the temperature", "steps": [("look up the temperature", "i am in san diego")]})

def replan(inputs):
    inputs['state'].past_steps.append((inputs['state'].plan[0], inputs['agent_outcome'].return_values['output']))
    sub_inputs = {
        "objective": inputs["objective"],
        "plan": inputs["state"].plan,
        "steps": inputs["state"].past_steps
    }
    output = replanner.invoke(sub_inputs)
    if isinstance(output, Response):
        inputs['state'].response = output.response
    else:
        inputs['state'].plan = output.steps
        inputs['input'] = inputs['state'].plan[0]
    return inputs


def should_end(inputs):
    if inputs['state'].response:
        return True
    else:
        return False

from langgraph.graph import END, Graph

workflow = Graph()

# Add the plan node
workflow.add_node("plan", plan)

# Add the agent node, we give it name `agent` which we will use later
workflow.add_node("agent", agent)
# Add the action node, we give it name `action` which we will use later
workflow.add_node("action", action)

# Add a replan node
workflow.add_node("replan", replan)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("plan")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we go back to replan
        "end": "replan"
    }
)

# From plan we go to agent
workflow.add_edge('plan', 'agent')

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    {
        # If `tools`, then we call the tool node.
        True: END,
        False: "agent",
    }
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
chain = workflow.compile()

for s in chain.stream({"objective": "what is leo dicaprios gf age raised to .34"}):
    print(s)